# Notebook para acesso ao Receita Data - dados de viajantes

In [1]:
import getpass
import pandas as pd
from datalake_serpro import handler_dl

In [2]:
user = getpass.getuser() + '@DATALAKE.SERPRO'
senha = getpass.getpass(f'Senha para kinit {user}:')

Senha para kinit 25052288840@DATALAKE.SERPRO:········


In [3]:
commands = f'echo "{senha}" | /opt/anaconda3/bin/kinit {user}'
! {commands}

Password for 25052288840@DATALAKE.SERPRO: 


In [4]:
! /opt/anaconda3/bin/klist

Ticket cache: FILE:/tmp/krb5cc_1227403730
Default principal: 25052288840@DATALAKE.SERPRO

Valid starting       Expires              Service principal
06/30/2021 14:20:40  07/01/2021 00:20:40  krbtgt/DATALAKE.SERPRO@DATALAKE.SERPRO
	renew until 07/07/2021 14:20:40


In [5]:
sql_dirf_2 = '''
SELECT vlrs.dd_cd_bene_cpf, vl_rend_tributavel

FROM dirf.wf_dirf_vlrs AS vlrs
JOIN dirf.wd_dirf_decl AS decl ON (decl.nr_decl = vlrs.nr_decl)
JOIN dirf.wd_dirf_decl_sit AS sit ON (decl.nr_decl_sit = sit.nr_dirf_decl_sit)
LEFT JOIN dime.wd_rc_receitas AS dime_receitas ON (vlrs.nr_bene_receita=dime_receitas.nr_receita)

WHERE vlrs.dd_cd_bene_cpf in (SELECT DISTINCT nr_impdr_expdr FROM coana.importacao_dsi WHERE dt_dia_regis > "2020-06-01")
AND vlrs.nr_fpag_dt_ano_calendario = 2020
AND vlrs.dd_cd_fpag_cnpj_est = '05526783000327'

'''

In [6]:
with handler_dl(DEBUG=True, timeout=300) as con:
    df_dirfs_auxilio_geral = pd.read_sql(sql_dirf_2, con)

Conectado ao Servidor impalaharfb.srv.datalake.serpro.


In [7]:
df_dirfs_auxilio_geral_sum = df_dirfs_auxilio_geral.groupby(['dd_cd_bene_cpf'])['vl_rend_tributavel'].sum().reset_index()

In [8]:
df_dirfs_auxilio_geral_sum

,dd_cd_bene_cpf,vl_rend_tributavel
0,00043226736,4200.0
1,00055245048,2400.0
2,00127758607,3900.0
3,00130934100,4200.0
4,00252028635,4200.0
...,...,...
692,98917021115,3900.0
693,99133890625,3844.0
694,99231425820,600.0
695,99448033591,4200.0


In [9]:
df_dirfs_auxilio_geral_sum.vl_rend_tributavel.describe()

count     697.000000
mean     3702.309900
std      1246.898334
min       600.000000
25%      3300.000000
50%      3900.000000
75%      4200.000000
max      8400.000000
Name: vl_rend_tributavel, dtype: float64

In [10]:
df_dirfs_auxilio_geral_sum.vl_rend_tributavel.sum()

2580510.0

In [11]:
df_dirfs_auxilio_geral.head()

,dd_cd_bene_cpf,vl_rend_tributavel
0,23695918896,0.0
1,66967350625,0.0
2,91252768591,600.0
3,39139462846,0.0
4,04486097700,0.0


In [12]:
len(set(df_dirfs_auxilio_geral.dd_cd_bene_cpf.values))

697

In [17]:
sql_dirf_3 = '''
SELECT vlrs.dd_cd_bene_cpf, vl_rend_tributavel

FROM dirf.wf_dirf_vlrs AS vlrs
JOIN dirf.wd_dirf_decl AS decl ON (decl.nr_decl = vlrs.nr_decl)
JOIN dirf.wd_dirf_decl_sit AS sit ON (decl.nr_decl_sit = sit.nr_dirf_decl_sit)
LEFT JOIN dime.wd_rc_receitas AS dime_receitas ON (vlrs.nr_bene_receita=dime_receitas.nr_receita)

WHERE vlrs.dd_cd_bene_cpf in (SELECT DISTINCT nr_impdr_expdr FROM coana.importacao_dsi WHERE dt_dia_regis > "2020-01-01")
AND vlrs.nr_fpag_dt_ano_calendario = 2020
AND vlrs.dd_cd_fpag_cnpj_est = '05526783000327'

'''

In [18]:
with handler_dl(DEBUG=True, timeout=300) as con:
    df_dirfs_auxilio_geral_ano_inteiro = pd.read_sql(sql_dirf_3, con)
    
df_dirfs_auxilio_geral_ano_sum = df_dirfs_auxilio_geral_ano_inteiro.groupby(['dd_cd_bene_cpf'])['vl_rend_tributavel'].sum().reset_index()


Conectado ao Servidor impalaharfb.srv.datalake.serpro.


In [19]:
df_dirfs_auxilio_geral_ano_sum.vl_rend_tributavel.sum()

4462501.0

In [20]:
df_dirfs_auxilio_geral_ano_sum.vl_rend_tributavel.describe()

count    1200.000000
mean     3718.750833
std      1299.755780
min       600.000000
25%      3300.000000
50%      3900.000000
75%      4200.000000
max      8400.000000
Name: vl_rend_tributavel, dtype: float64

In [23]:
print(f'{len(set(df_dirfs_auxilio_geral.dd_cd_bene_cpf.values))} pessoas que registraram DSI a partir de 01/06/2020 receberam DIRF do  Ministério da Cidadania – CNPJ: 05.526.783/0003-27.')
print(f'Foram pagos a estas pessoas R$ {df_dirfs_auxilio_geral_sum.vl_rend_tributavel.sum():,.2f} ')

697 pessoas que registraram DSI a partir de 01/06/2020 receberam DIRF do  Ministério da Cidadania – CNPJ: 05.526.783/0003-27.
Foram pagos a estas pessoas R$ 2,580,510.00 


In [24]:
print(f'{len(set(df_dirfs_auxilio_geral_ano_inteiro.dd_cd_bene_cpf.values))} pessoas registraram DSI a partir de 01/01/2020  receberam DIRF do  Ministério da Cidadania – CNPJ: 05.526.783/0003-27.')
print(f'Foram pagos a estas pessoas R$ {df_dirfs_auxilio_geral_ano_sum.vl_rend_tributavel.sum():,.2f} ')

1200 pessoas registraram DSI a partir de 01/01/2020  receberam DIRF do  Ministério da Cidadania – CNPJ: 05.526.783/0003-27.
Foram pagos a estas pessoas R$ 4,462,501.00 
